<a href="https://colab.research.google.com/github/batuhanyndny/notebooks/blob/master/scaler_coup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/batuhanyndny/scaler-coupling-constant

Cloning into 'scaler-coupling-constant'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 4), reused 26 (delta 3), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
cd scaler-coupling-constant/champs-scalar-coupling/

/content/scaler-coupling-constant/champs-scalar-coupling/scaler-coupling-constant/champs-scalar-coupling/scaler-coupling-constant/champs-scalar-coupling


In [ ]:
!ls

dipole_moments.csv		   scaler_coup.py
magnetic_shielding_tensors.csv	   structures.csv
mulliken_charges.csv		   structures.zip
potential_energy.csv		   test.csv
sample_submission.csv		   train.csv
scalar_coupling_contributions.csv


In [ ]:
!git lfs pull

Git LFS: (10 of 10 files) 954.53 MB / 954.53 MB


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

pd.set_option('display.expand_frame_repr', False)

dipole = pd.read_csv("dipole_moments.csv")
magn_she = pd.read_csv("magnetic_shielding_tensors.csv")
mulliken = pd.read_csv("mulliken_charges.csv")
potent_eng = pd.read_csv("potential_energy.csv")
coup_cont = pd.read_csv("scalar_coupling_contributions.csv")
struct = pd.read_csv("structures.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")


def map_atom_info(df, atom_idx):
    df = pd.merge(df, struct, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

In [ ]:
def splitData(splitNum, x, y, w):
  if x.shape[0] == y.shape[0] == w.shape[0]:
    print("X shape: ",x.shape[0])
    print("Y shape: ",y.shape[0])
    print("W shape: ",w.shape[0])
    out_count = int(x.shape[0]/splitNum)
    returnX = []
    returnY = []
    returnW = []
    index_X = 0
    index_Y = 0
    index_W = 0
    for i in range(1,splitNum+1):
      x_i = x[index_X:out_count*i,:]
      index_X = out_count*i
      returnX.append(x_i)
    for j in range(1,splitNum+1):
      y_j = y[index_Y:out_count*j]
      index_Y = out_count*j
      returnY.append(y_j)  
    for k in range(1,splitNum+1):
      w_k = w[index_W:out_count*k,:]
      index_W = out_count*k
      returnW.append(w_k)
    return returnX, returnY, returnW
  else:
    print("shapes doesn't match")
    print("X shape: ",x.shape[0])
    print("Y shape: ",y.shape[0])
    print("W shape: ",w.shape[0])

In [ ]:
def normalize(d):
     return (d-min(d))/(max(d)-min(d))

In [ ]:
#f(x) =  alpha * (exp(x) - 1.) for x < 0, f(x) = x for x >= 0
def ELU(alpha, x, b):
  if np.linalg.norm(x) > 0:
    return x 
  elif np.linalg.norm(x) == 0:
    x + b
  else:
    return alpha * (np.exp(x) - 1)

In [ ]:
def reshapeY(y_list):
  for i in range(len(y_list)):
    y_list[i] = y_list[i].values.reshape(-1,1)
  return y_list

In [ ]:
# In backward propagation we will use y_head that found in forward progation
# Therefore instead of writing backward propagation method, lets combine forward propagation and backward propagation
def forward_backward_propagation(w,b,x_train,y_train):
    # forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = ELU(0.06,z, b)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]      # x_train.shape[1]  is for scaling
    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return cost,gradients



In [ ]:
def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    # updating(learning) parameters is number_of_iterarion times
    for i in range(number_of_iterarion):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        # lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 10 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
    # we update(learn) parameters weights and bias
#     parameters = {"weight": w,"bias": b}
#     plt.plot(index,cost_list2)
#     plt.xticks(index,rotation='vertical')
#     plt.xlabel("Number of Iterarion")
#     plt.ylabel("Cost")
#     plt.show()
    return parameters, gradients, cost_list
#parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate = 0.009,number_of_iterarion = 200)

In [ ]:
train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
test_p_0 = test[['x_0', 'y_0', 'z_0']].values
test_p_1 = test[['x_1', 'y_1', 'z_1']].values

train['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test['dist'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)

train['dist_to_type_mean'] = train['dist'] / train.groupby('type')['dist'].transform('mean')
test['dist_to_type_mean'] = test['dist'] / test.groupby('type')['dist'].transform('mean')


columns = ["scalar_coupling_constant","dist_to_type_mean","dist"]

for i in columns:
    train_ = remove_outlier(train, "{}".format(i))


train = train.drop("id", axis=1)
train = train.drop("molecule_name", axis=1)


y = train.iloc[:,3]
y_ = pd.DataFrame(y.values)
y_["zeros"] = np.zeros(y.shape[0])

train = train.drop("scalar_coupling_constant", axis=1)

le = LabelEncoder()

train.iloc[:,2] = le.fit_transform(train.iloc[:,2])
train.iloc[:,3] = le.fit_transform(train.iloc[:,3])
train.iloc[:,7] = le.fit_transform(train.iloc[:,7])

ohe = OneHotEncoder(categorical_features = [2,3,7])

X = ohe.fit_transform(train).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state= 42 )

In [ ]:
normalize_index = 0
for i in x_train:
  x_train[normalize_index] = normalize(i)
  normalize_index += 1

In [ ]:
b = 0
w = normalize(np.random.rand(x_train.shape[0],1))

In [ ]:
x_,y_,w_ = splitData(1000,x_train,y_train,w)

X shape:  3260702
Y shape:  3260702
W shape:  3260702


In [ ]:
y_head = []
for i in range(len(x_)):
  y_head_i = np.sum(x_[i]*w_[i],axis=1)
  y_head.append(y_head_i)

In [ ]:
y_head[0].shape

(3260,)

In [ ]:
print("len of x_", len(x_))
print("len of y_", len(y_))
print("len of w_", len(w_))

len of x_ 1000
len of y_ 1000
len of w_ 1000


In [ ]:
print('Shape of x_[0]: ',x_[0].shape)
print('Shape of y_[0]: ',y_[0].shape)
print('Shape of w_[0]: ',w_[0].shape)

Shape of x_[0]:  (3260, 22)
Shape of y_[0]:  (3260,)
Shape of w_[0]:  (3260, 1)


In [ ]:
y_2 = reshapeY(y_)

In [ ]:
y_2[0]

array([[  3.43718],
       [-11.2845 ],
       [  3.87997],
       ...,
       [  6.1159 ],
       [  1.11769],
       [ -3.49233]])

In [ ]:
sigmoid(np.dot(w_[0].T,x_[0]))

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.]])

In [ ]:
parameters, gradients, cost_list = update(w_[i], b, x_[i], y_[i], learning_rate = 0.009,number_of_iterarion = 200)

Y_HEAD [[ 201.65833166  185.12341599  211.74911605  193.41853791  186.87714335
   219.93076557  197.26707345  188.51375614  296.21643321  265.24629702
   206.63965323  192.42239922 1597.81525973  803.39539077  202.17801394
   203.5409064   194.67821204  201.6139961   167.66091375  196.68434249
   447.18072526  296.06711505]]
Z :  [[ 201.65833166  185.12341599  211.74911605  193.41853791  186.87714335
   219.93076557  197.26707345  188.51375614  296.21643321  265.24629702
   206.63965323  192.42239922 1597.81525973  803.39539077  202.17801394
   203.5409064   194.67821204  201.6139961   167.66091375  196.68434249
   447.18072526  296.06711505]]
LOSS:  [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
Cost after iteration 0: nan
Y_HEAD [[-8931.43287569 -8917.28267341 -8944.32492727 ... -8925.49112849
  -9211.0665488  -9039.37969932]
 [-8942.6972513

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Y_HEAD [[263998.26000171 263573.68818261 264345.39405564 ... 263768.64136751
  271729.89170874 266970.08123123]
 [263999.13885113 263574.34095949 264346.34557112 ... 263769.70410441
  271733.15981102 266971.90580785]
 [263998.2335682  263573.66854877 264345.36543651 ... 263768.60940314
  271729.79341269 266970.02635269]
 ...
 [263998.10008848 263573.56940496 264345.22092026 ... 263768.44799457
  271729.29705304 266969.74923584]
 [263998.39846977 263573.79103156 264345.54397267 ... 263768.80880816
  271730.40661808 266970.36870434]
 [263998.67367704 263573.99544536 264345.84193496 ... 263769.14159887
  271731.43000786 266970.94006133]]
Z :  [[263998.26000171 263573.68818261 264345.39405564 ... 263768.64136751
  271729.89170874 266970.08123123]
 [263999.13885113 263574.34095949 264346.34557112 ... 263769.70410441
  271733.15981102 266971.90580785]
 [263998.2335682  263573.66854877 264345.36543651 ... 263768.60940314
  271729.79341269 266970.02635269]
 ...
 [263998.10008848 263573.5694049

KeyboardInterrupt: ignored

In [ ]:
y_[0].shape

In [ ]:
def update2(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    # updating(learning) parameters is number_of_iterarion times
    for i in range(number_of_iterarion):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation2(w,b,x_train,y_train)
        cost_list.append(cost)
        # lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 10 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
    # we update(learn) parameters weights and bias
    parameters = {"weight": w,"bias": b}
#     plt.plot(index,cost_list2)
#     plt.xticks(index,rotation='vertical')
#     plt.xlabel("Number of Iterarion")
#     plt.ylabel("Cost")
#     plt.show()
    return parameters, gradients, cost_list
#parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate = 0.009,number_of_iterarion = 200)

In [ ]:
# In backward propagation we will use y_head that found in forward progation
# Therefore instead of writing backward propagation method, lets combine forward propagation and backward propagation
def forward_backward_propagation2(w,b,x_train,y_train):
    # forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = ELU(0.06,z, b)
    print("y_head: ",y_head)
    print("np.log(y_head): ",np.log(y_head))
    print("1-y_head: ",1-y_head)
    print("np.log(1-y_head)", np.log(1-y_head))
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]      # x_train.shape[1]  is for scaling
    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return cost,gradients


In [ ]:
parameters, gradients, cost_list = update2(w_[1], b, x_[1], y_[1], learning_rate = 0.009,number_of_iterarion = 1)

y_head:  [[ 206.91780361  189.98032373  216.63962126  198.40635266  191.98261085
   223.70803137  202.35229887  193.34894624  299.99997514  269.1694524
   211.71064961  197.21587697 1595.71283165  801.24114233  203.81738048
   201.35098089  199.23793161  202.36542685  167.11751778  196.88035331
   447.84465324  299.71610124]]
np.log(y_head):  [[5.33232163 5.24692051 5.37823524 5.29031721 5.2574048  5.41034177
  5.31001023 5.26449657 5.70378239 5.59534112 5.35522048 5.28429895
  7.37507583 6.68616195 5.3172244  5.30504956 5.29449975 5.31007511
  5.11869726 5.2825962  6.10444642 5.7028357 ]]
1-y_head:  [[ -205.91780361  -188.98032373  -215.63962126  -197.40635266
   -190.98261085  -222.70803137  -201.35229887  -192.34894624
   -298.99997514  -268.1694524   -210.71064961  -196.21587697
  -1594.71283165  -800.24114233  -202.81738048  -200.35098089
   -198.23793161  -201.36542685  -166.11751778  -195.88035331
   -446.84465324  -298.71610124]]
np.log(1-y_head) [[nan nan nan nan nan nan nan n

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':


In [ ]:
np.log(1-y_[1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


array([[2.50798007],
       [       nan],
       [       nan],
       ...,
       [       nan],
       [       nan],
       [       nan]])

In [ ]:
np.log(y_[1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


array([[       nan],
       [1.10075   ],
       [4.43148324],
       ...,
       [1.38319958],
       [2.63101076],
       [1.54128754]])

In [ ]:
1-y_[1]

array([[ 12.2801 ],
       [ -2.00642],
       [-83.056  ],
       ...,
       [ -2.98764],
       [-12.8878 ],
       [ -3.6706 ]])